In [ ]:
import pandas as pd

In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/Research_SEM4/Dataset_AfterClean/test_df.csv")

In [ ]:
columns_to_delete = ['Unnamed: 0']

test_df = test_df.drop(columns=columns_to_delete)
test_df.head()

,original_text,extractive_summary
0,jakarta kejatuhan abdurrahman wahid dari k...,ia juga tak mendapat dukungan dari militer be...
1,jakarta mantan presiden abdurrahman wahid ...,jakarta mantan presiden abdurrahman wahid ...
2,jakarta bank muamalat bergeming ancaman l...,bahkan bank muamalat berencana meningkatkan k...
3,jakarta secara keseluruhan lawatan preside...,jakarta secara keseluruhan lawatan preside...
4,jakarta sejumlah prajurit tni rabu 03 10...,saat memberi pengarahan widodo mengatakan se...


In [ ]:
!pip install datasets transformers torch accelerate

In [ ]:
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd
import re
from transformers import BertTokenizer, BertForTokenClassification
from datasets import load_dataset, Dataset
from transformers import Trainer, TrainingArguments

In [ ]:
device = 'cuda'

In [ ]:
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
model_path = "/content/drive/MyDrive/Research_SEM4/Hasil_FineTuning/mBert:Extractive"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForTokenClassification.from_pretrained(model_path).to(device)
model.eval()

In [ ]:
from transformers import BertTokenizer, BertForTokenClassification
import torch
from tqdm import tqdm

test_dataset = Dataset.from_pandas(test_df)

def generate_extractive_summaries(model, tokenizer, dataset, device):
    model.eval()
    summaries = []

    for example in tqdm(dataset, desc="Generating Extractive Summaries"):
        text = example["original_text"]
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1).squeeze().cpu().numpy()

        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().cpu().numpy())

        # Build summary by including tokens predicted as label 1
        summary_tokens = [
            token for token, pred in zip(tokens, predictions)
            if pred == 1 and token not in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]
        ]

        # Reconstruct text
        summary = tokenizer.convert_tokens_to_string(summary_tokens)
        summaries.append(summary.strip())

    return summaries

In [ ]:
# Assuming test_dataset is a Dataset object like train_dataset/dev_dataset
predicted_summaries = generate_extractive_summaries(model, tokenizer, test_dataset, device)


Generating Extractive Summaries: 100%|██████████| 3032/3032 [01:12<00:00, 42.11it/s]


In [ ]:
result_df = pd.DataFrame({
    "original_text": test_df["original_text"],
    "true_extractive_summary": test_df["extractive_summary"],
    "predicted_extractive_summary": predicted_summaries
})

# View in notebook
result_df.head()


,original_text,true_extractive_summary,predicted_extractive_summary
0,jakarta kejatuhan abdurrahman wahid dari k...,ia juga tak mendapat dukungan dari militer be...,kejatuhan wahid dari ke untuk dukungan di parl...
1,jakarta mantan presiden abdurrahman wahid ...,jakarta mantan presiden abdurrahman wahid ...,jakarta mantan presiden abdurrahman wahid meni...
2,jakarta bank muamalat bergeming ancaman l...,bahkan bank muamalat berencana meningkatkan k...,bank muamalat bank bank yang takannya tak me b...
3,jakarta secara keseluruhan lawatan preside...,jakarta secara keseluruhan lawatan preside...,jak secara presiden megawati ke dan dalam kunj...
4,jakarta sejumlah prajurit tni rabu 03 10...,saat memberi pengarahan widodo mengatakan se...,jakarta sejumlah prajurit tni rabu 03 10 mengi...


In [ ]:
result_df.to_csv("mbert_extractive_summary_predictions.csv", index=False)
print("Saved predictions to mbert_extractive_summary_predictions.csv")


Saved predictions to mbert_extractive_summary_predictions.csv


In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=341dec7fd4af4b65f1c76e72a9d4aa53ce856784f2bfaef2dfca77964a55a0aa
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer

# Siapkan referensi dan kandidat
references = test_df['extractive_summary'].tolist()
candidates = result_df['predicted_extractive_summary'].tolist()

# Inisialisasi Rouge Scorer dengan ROUGE-L
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# List untuk menyimpan skor
rouge1_precisions = []
rouge1_recalls = []
rouge1_f1s = []

rouge2_precisions = []
rouge2_recalls = []
rouge2_f1s = []

rougeL_precisions = []
rougeL_recalls = []
rougeL_f1s = []

# Iterasi untuk setiap pasang kandidat dan referensi
for candidate, reference in zip(candidates, references):
    scores = scorer.score(reference, candidate)

    # ROUGE-1
    rouge1_precisions.append(scores['rouge1'].precision)
    rouge1_recalls.append(scores['rouge1'].recall)
    rouge1_f1s.append(scores['rouge1'].fmeasure)

    # ROUGE-2
    rouge2_precisions.append(scores['rouge2'].precision)
    rouge2_recalls.append(scores['rouge2'].recall)
    rouge2_f1s.append(scores['rouge2'].fmeasure)

    # ROUGE-L
    rougeL_precisions.append(scores['rougeL'].precision)
    rougeL_recalls.append(scores['rougeL'].recall)
    rougeL_f1s.append(scores['rougeL'].fmeasure)

# Cetak hasil agregasi
print(f"ROUGE-1 Precision: {sum(rouge1_precisions) / len(rouge1_precisions):.4f}")
print(f"ROUGE-1 Recall: {sum(rouge1_recalls) / len(rouge1_recalls):.4f}")
print(f"ROUGE-1 F1: {sum(rouge1_f1s) / len(rouge1_f1s):.4f}")

print(f"\nROUGE-2 Precision: {sum(rouge2_precisions) / len(rouge2_precisions):.4f}")
print(f"ROUGE-2 Recall: {sum(rouge2_recalls) / len(rouge2_recalls):.4f}")
print(f"ROUGE-2 F1: {sum(rouge2_f1s) / len(rouge2_f1s):.4f}")

print(f"\nROUGE-L Precision: {sum(rougeL_precisions) / len(rougeL_precisions):.4f}")
print(f"ROUGE-L Recall: {sum(rougeL_recalls) / len(rougeL_recalls):.4f}")
print(f"ROUGE-L F1: {sum(rougeL_f1s) / len(rougeL_f1s):.4f}")


ROUGE-1 Precision: 0.2710
ROUGE-1 Recall: 0.5395
ROUGE-1 F1: 0.3535

ROUGE-2 Precision: 0.1931
ROUGE-2 Recall: 0.3883
ROUGE-2 F1: 0.2528

ROUGE-L Precision: 0.2620
ROUGE-L Recall: 0.5217
ROUGE-L F1: 0.3418
